# Problem 1 – Binary Words and Operations  
**Author:** Michael Ferry  
**Date:** October 2025 

In [1]:
import numpy as np


## 32-bit Integers

In SHA-256 everything uses 32-bit numbers.  
Python doesn’t do that by default, so I made a small helper that turns any number into a 32-bit one.  
This makes sure the maths works the same way as in the standard.



I used the modulo (%) operator so the number always stays in the 32-bit range (0 to 4294967295).  
If it goes below 0 or above the limit, it wraps back around like a loop.  
This makes it act the same way as real 32-bit hardware does in SHA-256.

In [2]:
def to_uint32(x):

    return np.uint32(x % (1 << 32))

# Just runs a quick test to check the wrap
print(to_uint32(123))   # normal
print(to_uint32(-1))    # wraps around to 4294967295

123
4294967295


## Parity Function  

Parity checks three 32-bit numbers and returns 1 in places where an odd number of bits are 1.  
I used the XOR (^) operator because it flips bits this way and works perfectly for this rule.

In [3]:
def Parity(x, y, z):
    """Gives 1 for each bit where an odd number of x, y, z bits are 1."""
    x, y, z = map(to_uint32, (x, y, z))
    return x ^ y ^ z

# Just runs a quick test of the parity function
print(bin(Parity(0b1010, 0b0101, 0b0011)))


0b1100


## Choice Function

The Choice function checks three 32-bit numbers and returns bits from **y** or **z** depending on **x**.  
If a bit in **x** is 1, the corresponding bit from **y** is chosen.  
If it’s 0, the bit from **z** is chosen.  


In [ ]:
def Ch(x, y, z):
    """Returns bits from y where x has 1s otherwise from z."""
    x, y, z = map(to_uint32, (x, y, z))
    return (x & y) ^ (~x & z)

# Runs a test of the choice function
print(bin(Ch(0b1010, 0b1100, 0b0110)))


0b1100


## Majority Function

The Majority function checks three 32-bit numbers and returns 1 in each bit position where two or more of the bits are 1.  

This means the result bit is 1 if at least two of x, y, z have 1 in that position.


In [9]:
def Maj(x, y, z):
    """Gives 1 for each bit where at least two of x, y, z bits are 1."""
    x, y, z = map(to_uint32, (x, y, z))
    return (x & y) ^ (x & z) ^ (y & z)

# Just runs a quick test of the majority function
print(bin(Maj(0b1010, 0b1100, 0b0110)))

0b1110
